# Change random character in each attribute

## Attacking function for all attributes

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import sys
sys.path.append('../')

In [14]:
import random as rd
import string
import pandas as pd
import deepmatcher as dm
import numpy as np

In [4]:
def changeRandomCharacter(att,permittedPositions):
    random_char = rd.choice(string.ascii_letters+string.digits+string.punctuation)
    new_char_idx = rd.choice(permittedPositions)
    res = att[:new_char_idx]+random_char+att[new_char_idx+1:]
    return res

In [5]:
def attackDataset(dataset,ignore_columns):
    for col in list(dataset):
        if col not in ignore_columns:
            dataset[col] = dataset[col].apply(lambda s: changeRandomCharacter(str(s),
                                                                             np.arange(len(str(s)))))

In [10]:
test_df = pd.read_csv('../Structured/itunes-amazon/merged_test.csv')
test_df.head()

,label,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released,id
0,0,Elevator ( feat . Timbaland ),Flo Rida,Mail On Sunday ( Deluxe Version ),"Hip-Hop/Rap , Music , Dirty South",$ 1.99,2008 Atlantic Recording Corporation for the Un...,3:55,17-Mar-08,Money Right ( feat . Rick Ross & Brisco ) [ Ex...,Flo Rida,Mail On Sunday [ Explicit ],Rap & Hip-Hop,$ 1.29,2013 Warner Bros. . Records Inc.,3:17,"March 17 , 2008",731
1,0,The Woodland Realm ( Extended Version ),Howard Shore,The Hobbit : The Desolation of Smaug ( Origina...,"Soundtrack , Music , Soundtrack , Classical , ...",$ 1.29,‰ ãÑ 2013 WaterTower Music,5:14,10-Dec-13,The High Fells ( Extended Version ),Howard Shore,The Hobbit : The Desolation of Smaug ( Origina...,Soundtracks,$ 1.29,2013 WaterTower Music / Warner Bros. . Enterta...,3:38,"December 10 , 2013",672
2,1,Extra Extra Credit,Wiz Khalifa,Flight School,"Hip-Hop/Rap , Music",$ 0.99,2009 Rostrum Records,4:03,17-Apr-09,Extra Extra Credit [ Explicit ],Wiz Khalifa,Flight School [ Explicit ],Rap & Hip-Hop,$ 0.99,2013 Mad Decent,4:03,"April 17 , 2009",669
3,0,Toyfriend ( feat . Wynter Gordon ) [ Continuou...,David Guetta,One Love ( Deluxe Version ),"Dance , Music",$ 1.29,2010 Gum Prod licence exclusive Parlophone Mus...,2:51,21-Aug-09,Sound Of Letting Go ( Feat . Chris Willis ),David Guetta,One Love ( Deluxe Version ),Dance & Electronic,$ 1.29,( C ) 2014 Swedish House Mafia Holdings Ltd ( ...,3:47,"August 21 , 2009",734
4,0,Dangerous ( feat . Sam Martin ) [ Robin Schulz...,David Guetta,Listen ( Deluxe Version ),"Dance , Music , Rock , House , Electronic , Fr...",$ 1.29,2014 What A Music Ltd. under exclusive license...,3:20,24-Nov-14,Missing You ( Feat . Novel ; Continuous Mix Ve...,David Guetta,One Love ( Deluxe Version ),Dance & Electronic,$ 1.29,( C ) 2014 Swedish House Mafia Holdings Ltd ( ...,4:59,"August 21 , 2009",699


In [ ]:
attackDataset(test_df,['label','id'])

In [ ]:
test_df.head()

In [ ]:
test_df.to_csv('../Structured/itunes-amazon/attacked_test.csv',index=False)

### ITunes-Amazon

In [ ]:
train,test = dm.data.process('../Structured/itunes-amazon',train='merged_train.csv',
                       test='merged_test.csv',left_prefix='ltable_',
                      right_prefix='rtable_')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('../models/itunes_amazon_hybrid.pth')

In [ ]:
model.run_eval(test)

In [4]:
train,validation,attacked_test = dm.data.process('../Structured/itunes-amazon/',train='merged_train.csv',
                                                validation='merged_valid.csv',test='attacked_test.csv',
                                                left_prefix='ltable_',right_prefix='rtable_')

In [5]:
model2 = dm.MatchingModel(attr_summarizer='hybrid')
model2.run_train(train,validation,pos_neg_ratio=5,best_save_path='../models/itunes_amazon_forattack.pth',
              epochs=20)

* Number of trainable parameters: 17757810
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:    4.3 | Load Time:    1.2 || F1:  37.06 | Prec:  23.53 | Rec:  87.18 || Ex/s:  57.62

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.5 | Load Time:    0.4 || F1:  39.71 | Prec:  24.77 | Rec: 100.00 || Ex/s: 120.95

* Best F1: 39.705882352941174
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:    5.0 | Load Time:    1.3 || F1:  53.05 | Prec:  36.82 | Rec:  94.87 || Ex/s:  50.62

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.5 | Load Time:    0.4 || F1:  53.76 | Prec:  37.88 | Rec:  92.59 || Ex/s: 122.62

* Best F1: 53.76344086021504
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:    4.9 | Load Time:    1.4 || F1:  66.67 | Prec:  53.49 | Rec:  88.46 || Ex/s:  51.35

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    0.5 | Load Time:    0.4 || F1:  55.81 | Prec:  40.68 | Rec:  88

81.25

In [6]:
model2.run_eval(attacked_test)

===>  EVAL Epoch 7
Finished Epoch 7 || Run Time:    0.6 | Load Time:    0.5 || F1:  55.07 | Prec:  45.24 | Rec:  70.37 || Ex/s:  99.35



55.07246376811594

### Beer

In [14]:
test_df = pd.read_csv('../Structured/Beer/merged_test.csv')
test_df.head()

,label,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV,id
0,0,Bulleit Bourbon Barrel Aged G'Knight,Oskar Blues Grill & Brew,American Amber / Red Ale,8.70 %,Figure Eight Bourbon Barrel Aged Jumbo Love,Figure Eight Brewing,Barley Wine,-,642
1,0,Double Dragon Imperial Red Ale,Phillips Brewing Company,American Amber / Red Ale,8.20 %,Scuttlebutt Mateo Loco Imperial Red Ale,Scuttlebutt Brewing Co. .,American Strong Ale,7.10 %,560
2,0,Batch 200 _ `` Olympus Mons Imperial Red Ale,Due South Brewing Co. .,American Amber / Red Ale,9.20 %,Scuttlebutt Mateo Loco Imperial Red Ale,Scuttlebutt Brewing Co. .,American Strong Ale,7.10 %,562
3,1,Honey Basil Amber,Rude Hippo Brewing Company,American Amber / Red Ale,7.40 %,Rude Hippo Honey Basil Amber,18th Street Brewery,Amber Ale,7.40 %,598
4,0,Clifford 's Big Red Ale,Funky Buddha Brewery,American Amber / Red Ale,9.70 %,Big Island Red Giant Ale,Big Island Brewhaus,American Strong Ale,6.50 %,593


In [15]:
attackDataset(test_df,['id','label'])
test_df.head()

,label,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV,id
0,0,Bu%leit Bourbon Barrel Aged G'Knight,Oskar BlXes Grill & Brew,Ameri!an Amber / Red Ale,8.70?%,Figure Eight Bourbon B/rrel Aged Jumbo Love,Figure Eight grewing,Barl+y Wine,z,642
1,0,Double Dragon Imp4rial Red Ale,Phillips BrFwing Company,American Amb!r / Red Ale,Y.20 %,Scuttlebutt M0teo Loco Imperial Red Ale,Scuttlebutt Brewing Co> .,American S/rong Ale,7.10 w,560
2,0,Batch 200 _ `` Olympus Mons Imperial Re' Ale,Due South Brewing Cf. .,American Amber / 5ed Ale,9.2j %,Scuttlebutt Mateo_Loco Imperial Red Ale,Scuttlebutt Brewing Co. *,AmeriMan Strong Ale,7.10 j,562
3,1,Honey Ba-il Amber,Rude HippC Brewing Company,AmericanCAmber / Red Ale,7.40 #,RudeEHippo Honey Basil Amber,18th<Street Brewery,Ambzr Ale,7.4+ %,598
4,0,Cliffkrd 's Big Red Ale,Funky Buddh1 Brewery,Am<rican Amber / Red Ale,9.77 %,Big Island Red GiaQt Ale,Big Island Bjewhaus,American St?ong Ale,6H50 %,593


In [16]:
test_df.to_csv('../Structured/Beer/attacked_test.csv',index=False)

In [18]:
train,validation,attacked_test = dm.data.process('../Structured/Beer/',train='merged_train.csv',
                                                validation='merged_valid.csv',test='attacked_test.csv',
                                                left_prefix='ltable_',right_prefix='rtable_',cache='attack.pth')

In [20]:
model2 = dm.MatchingModel(attr_summarizer='hybrid')
model2.run_train(train,validation,pos_neg_ratio=5,best_save_path='../models/beer_hybrid_forattack.pth',
              epochs=20)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:    2.1 | Load Time:    0.5 || F1:  27.67 | Prec:  18.49 | Rec:  55.00 || Ex/s: 103.25

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.1 || F1:  26.67 | Prec:  15.38 | Rec: 100.00 || Ex/s: 254.67

* Best F1: 26.666666666666668
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:    2.1 | Load Time:    0.5 || F1:  34.17 | Prec:  21.38 | Rec:  85.00 || Ex/s: 103.79

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.2 | Load Time:    0.1 || F1:  47.83 | Prec:  34.38 | Rec:  78.57 || Ex/s: 255.05

* Best F1: 47.826086956521735
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:    2.1 | Load Time:    0.5 || F1:  65.00 | Prec:  48.75 | Rec:  97.50 || Ex/s: 104.59

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    0.2 | Load Time:    0.1 || F1:  40.00 | Prec:  25.00 | Rec: 100

70.27027027027026

In [21]:
model2.run_eval(attacked_test)

===>  EVAL Epoch 11
Finished Epoch 11 || Run Time:    0.2 | Load Time:    0.2 || F1:  52.00 | Prec:  36.11 | Rec:  92.86 || Ex/s: 239.85



52.00000000000001

### DBLP-ACM

In [40]:
test_df = pd.read_csv('../Structured/DBLP-ACM/merged_test.csv')
attackDataset(test_df,['id','label'])
test_df.to_csv('../Structured/DBLP-ACM/attacked_test.csv',index=False)

In [11]:
train,validation,test = dm.data.process('../Structured/DBLP-ACM',train='merged_train.csv',
                                        validation='merged_validation.csv',
                       test='attacked_test.csv',left_prefix='ltable_',
                      right_prefix='rtable_',cache='attack.pth')


Reading and processing data from "../Structured/DBLP-ACM/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/DBLP-ACM/merged_validation.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/DBLP-ACM/attacked_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [12]:
model2 = dm.MatchingModel(attr_summarizer='hybrid')
model2.run_train(train,validation,pos_neg_ratio=6,best_save_path='../models/dblpacm_hybrid_forattack.pth',
              epochs=10)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:46


Finished Epoch 1 || Run Time:   38.7 | Load Time:    8.5 || F1:  77.43 | Prec:  63.78 | Rec:  98.50 || Ex/s: 156.91

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 1 || Run Time:    4.0 | Load Time:    2.7 || F1:  95.59 | Prec:  91.55 | Rec: 100.00 || Ex/s: 369.88

* Best F1: 95.58665231431647
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:49


Finished Epoch 10 || Run Time:   40.7 | Load Time:    9.0 || F1:  99.40 | Prec:  98.81 | Rec: 100.00 || Ex/s: 149.18

===>  EVAL Epoch 10


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 10 || Run Time:    4.2 | Load Time:    2.7 || F1:  98.55 | Prec:  97.36 | Rec:  99.77 || Ex/s: 360.75

---------------------

Loading best model...
Training done.


98.7736900780379

In [13]:
model2.run_eval(test)

===>  EVAL Epoch 6


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 6 || Run Time:    4.3 | Load Time:    2.9 || F1:  32.77 | Prec:  94.62 | Rec:  19.82 || Ex/s: 345.25



32.774674115456236

### Walmart-Amazon

In [12]:
test_df = pd.read_csv('../Structured/Walmart-Amazon/merged_test.csv')
attackDataset(test_df,['id','label'])

In [13]:
test_df.to_csv('../Structured/Walmart-Amazon/attacked_test.csv',index=False)

In [18]:
train,validation,attackec_test = dm.data.process('../Structured/Walmart-Amazon/',train='merged_train.csv',
                                        validation='merged_valid.csv',test='attacked_test.csv'
                                                 ,left_prefix='ltable_',right_prefix='rtable_',cache='attack.pth')


Reading and processing data from "../Structured/Walmart-Amazon/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Walmart-Amazon/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Walmart-Amazon/attacked_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [19]:
model2 = dm.MatchingModel(attr_summarizer='hybrid')
model2.run_train(train,validation,pos_neg_ratio=10,best_save_path='../models/walmartamazon_hybrid_forattack.pth',
              epochs=20)

* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 1 || Run Time:   29.5 | Load Time:    4.9 || F1:  20.69 | Prec:  11.62 | Rec:  94.27 || Ex/s: 179.08

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    3.5 | Load Time:    1.5 || F1:  24.78 | Prec:  14.27 | Rec:  93.78 || Ex/s: 409.37

* Best F1: 24.777549623545514
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 20 || Run Time:   29.4 | Load Time:    4.9 || F1:  99.14 | Prec:  98.29 | Rec: 100.00 || Ex/s: 179.35

===>  EVAL Epoch 20


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 20 || Run Time:    3.5 | Load Time:    1.5 || F1:  62.50 | Prec:  56.49 | Rec:  69.95 || Ex/s: 411.40

---------------------

Loading best model...
Training done.


65.83541147132169

In [21]:
model2.run_eval(attackec_test)

===>  EVAL Epoch 9


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 9 || Run Time:    3.7 | Load Time:    1.8 || F1:   9.57 | Prec:  29.73 | Rec:   5.70 || Ex/s: 371.92



9.565217391304346

### Amazon Google

In [33]:
test_df = pd.read_csv('../Structured/Amazon-Google/merged_test.csv')
attackDataset(test_df,['id','label'])
test_df.to_csv('../Structured/Amazon-Google/attacked_test.csv',index=False)

In [9]:
train,validation,attacked_test = dm.data.process('../Structured/Amazon-Google/',train='merged_train.csv',
                                        validation='merged_valid.csv',test='attacked_test.csv'
                                                 ,left_prefix='ltable_',right_prefix='rtable_',cache='attack.pth')


Reading and processing data from "../Structured/Amazon-Google/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Amazon-Google/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Amazon-Google/attacked_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [11]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=10,best_save_path='../models/amazongoogle_hybrid_forattack.pth',
              epochs=20)

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:24


Finished Epoch 1 || Run Time:   20.6 | Load Time:    3.5 || F1:  23.84 | Prec:  13.67 | Rec:  92.99 || Ex/s: 285.11

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    2.6 | Load Time:    1.1 || F1:  29.59 | Prec:  17.62 | Rec:  92.31 || Ex/s: 609.97

* Best F1: 29.589041095890412
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:24


Finished Epoch 20 || Run Time:   21.1 | Load Time:    3.6 || F1:  95.82 | Prec:  91.97 | Rec: 100.00 || Ex/s: 278.42

===>  EVAL Epoch 20


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 20 || Run Time:    2.5 | Load Time:    1.1 || F1:  68.07 | Prec:  66.94 | Rec:  69.23 || Ex/s: 642.05

---------------------

Loading best model...
Training done.


72.4202626641651

In [12]:
model.run_eval(attacked_test)

===>  EVAL Epoch 9


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 9 || Run Time:    2.3 | Load Time:    1.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 559.08



0.0

## Attack single random attribute
by changing one character on the left and on the right attribute

In [7]:
from utils.dataset_parser import getAttributes

In [8]:
## given two string calculate the indexes of commons characters
def getIntersectionIndexes(s1,s2):
    commonChars = set(s1).intersection(set(s2))
    commonCharsIndexes = [[],[]]
    commonCharsIndexes[0] = list(map(lambda c:s1.find(c),commonChars))
    commonCharsIndexes[1] = list(map(lambda c:s2.find(c),commonChars))
    if len(commonChars)==0:
        commonCharsIndexes[0] = np.arange(len(s1))
        commonCharsIndexes[1] = np.arange(len(s2))
    return commonCharsIndexes

In [9]:
def attackRandomAttribute(dataset,ignore_columns,leftPrefix='ltable_',rightPrefix='rtable_'):
    attributes = getAttributes(dataset,notAttributes=ignore_columns)
    dataset_index = dataset.index
    changedAttributes = []
    for idx in dataset_index:
        selectAttr = rd.choice(attributes)
        changedAttributes.append(selectAttr)
        lval = str(dataset.loc[idx][leftPrefix+selectAttr])
        rval = str(dataset.loc[idx][rightPrefix+selectAttr])
        permittedPositions = getIntersectionIndexes(lval,rval)
        dataset.at[idx,leftPrefix+selectAttr] = changeRandomCharacter(lval,permittedPositions[0])
        dataset.at[idx,rightPrefix+selectAttr] = changeRandomCharacter(rval,permittedPositions[1])
    dataset['attackedAttribute'] = changedAttributes

### Itunes-Amazon

In [12]:
test_df = pd.read_csv('../Structured/itunes-amazon/merged_test.csv')
attackRandomAttribute(test_df,ignore_columns=['id','label'])
test_df.to_csv('../Structured/itunes-amazon/attacked_test_singleattr.csv',index=False)

In [13]:
train,validation,attacked_test = dm.data.process('../Structured/itunes-amazon',train='merged_train.csv',
                       validation='merged_valid.csv',test='attacked_test_singleattr.csv',left_prefix='ltable_',
                      right_prefix='rtable_',cache='attack_single.pth')


Reading and processing data from "../Structured/itunes-amazon/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/itunes-amazon/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/itunes-amazon/attacked_test_singleattr.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [14]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=5,best_save_path='../models/itunes_amazon_singleattack.pth',
              epochs=20)


* Number of trainable parameters: 17757810
===>  TRAIN Epoch 1


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    4.5 | Load Time:    0.7 || F1:  37.80 | Prec:  23.76 | Rec:  92.31 || Ex/s:  61.76

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.4 | Load Time:    0.2 || F1:  40.91 | Prec:  25.71 | Rec: 100.00 || Ex/s: 185.76

* Best F1: 40.90909090909091
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 2 || Run Time:    3.3 | Load Time:    0.6 || F1:  39.39 | Prec:  24.53 | Rec: 100.00 || Ex/s:  81.45

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.4 | Load Time:    0.2 || F1:  42.28 | Prec:  27.08 | Rec:  96.30 || Ex/s: 183.84

* Best F1: 42.27642276422764
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3


...

0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 19 || Run Time:    3.2 | Load Time:    0.6 || F1:  93.98 | Prec:  88.64 | Rec: 100.00 || Ex/s:  82.83

===>  EVAL Epoch 19
Finished Epoch 19 || Run Time:    0.4 | Load Time:    0.2 || F1:  80.00 | Prec:  68.42 | Rec:  96.30 || Ex/s: 185.89

---------------------

===>  TRAIN Epoch 20


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 20 || Run Time:    3.2 | Load Time:    0.6 || F1:  93.98 | Prec:  88.64 | Rec: 100.00 || Ex/s:  82.78

===>  EVAL Epoch 20
Finished Epoch 20 || Run Time:    0.4 | Load Time:    0.2 || F1:  80.00 | Prec:  68.42 | Rec:  96.30 || Ex/s: 185.49

---------------------

Loading best model...
Training done.


81.25

In [15]:
model.run_eval(attacked_test)

===>  EVAL Epoch 13
Finished Epoch 13 || Run Time:    0.4 | Load Time:    0.2 || F1:  76.47 | Prec:  63.41 | Rec:  96.30 || Ex/s: 172.43



76.47058823529412

### Beer

In [16]:
test_df = pd.read_csv('../Structured/Beer/merged_test.csv')
attackRandomAttribute(test_df,ignore_columns=['id','label'])
test_df.to_csv('../Structured/Beer/attacked_test_singleattr.csv',index=False)

In [17]:
train,validation,attacked_test = dm.data.process('../Structured/Beer',train='merged_train.csv',
                       validation='merged_valid.csv',test='attacked_test_singleattr.csv',left_prefix='ltable_',
                      right_prefix='rtable_',cache='attack_single.pth')


Reading and processing data from "../Structured/Beer/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Beer/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Beer/attacked_test_singleattr.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [18]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=5,best_save_path='../models/itunes_amazon_singleattack.pth',
              epochs=20)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    1.2 | Load Time:    0.2 || F1:  24.21 | Prec:  15.33 | Rec:  57.50 || Ex/s: 196.49

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 455.41

* Best F1: 0.0
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    1.2 | Load Time:    0.2 || F1:  52.83 | Prec:  42.42 | Rec:  70.00 || Ex/s: 192.73

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.1 | Load Time:    0.1 || F1:  40.00 | Prec:  25.00 | Rec: 100.00 || Ex/s: 444.68

* Best F1: 40.0
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 19 || Run Time:    1.3 | Load Time:    0.2 || F1:  98.77 | Prec:  97.56 | Rec: 100.00 || Ex/s: 175.90

===>  EVAL Epoch 19
Finished Epoch 19 || Run Time:    0.1 | Load Time:    0.1 || F1:  68.57 | Prec:  57.14 | Rec:  85.71 || Ex/s: 405.04

---------------------

===>  TRAIN Epoch 20


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 20 || Run Time:    1.2 | Load Time:    0.2 || F1:  98.77 | Prec:  97.56 | Rec: 100.00 || Ex/s: 184.34

===>  EVAL Epoch 20
Finished Epoch 20 || Run Time:    0.2 | Load Time:    0.1 || F1:  68.57 | Prec:  57.14 | Rec:  85.71 || Ex/s: 393.39

---------------------

Loading best model...
Training done.


80.0

In [19]:
model.run_eval(attacked_test)

===>  EVAL Epoch 6
Finished Epoch 6 || Run Time:    0.1 | Load Time:    0.1 || F1:  66.67 | Prec:  54.55 | Rec:  85.71 || Ex/s: 414.49



66.66666666666666

### DBLP-ACM

In [30]:
test_df = pd.read_csv('../Structured/DBLP-ACM/merged_test.csv')
test_df['ltable_year'] = test_df['ltable_year'].astype(str)
test_df['rtable_year'] = test_df['rtable_year'].astype(str)
attackRandomAttribute(test_df,ignore_columns=['id','label'])
test_df.to_csv('../Structured/DBLP-ACM/attacked_test_singleattr.csv',index=False)

In [35]:
train,validation,attacked_test = dm.data.process('../Structured/DBLP-ACM',train='merged_train.csv',
                       validation='merged_validation.csv',test='attacked_test_singleattr.csv',left_prefix='ltable_',
                      right_prefix='rtable_',cache='attack_single.pth')


Reading and processing data from "../Structured/DBLP-ACM/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/DBLP-ACM/merged_validation.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/DBLP-ACM/attacked_test_singleattr.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [36]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=10,best_save_path='../models/dblpacm_hybrid_singleattack.pth',
              epochs=10)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:46


Finished Epoch 1 || Run Time:   38.5 | Load Time:    8.6 || F1:  69.98 | Prec:  54.05 | Rec:  99.25 || Ex/s: 157.45

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 1 || Run Time:    4.0 | Load Time:    2.7 || F1:  94.86 | Prec:  90.41 | Rec:  99.77 || Ex/s: 368.65

* Best F1: 94.86081370449678
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:50


Finished Epoch 10 || Run Time:   41.4 | Load Time:    9.3 || F1:  99.40 | Prec:  98.81 | Rec: 100.00 || Ex/s: 146.46

===>  EVAL Epoch 10


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 10 || Run Time:    4.5 | Load Time:    2.9 || F1:  98.23 | Prec:  96.72 | Rec:  99.77 || Ex/s: 337.41

---------------------

Loading best model...
Training done.


98.33518312985572

In [37]:
model.run_eval(attacked_test)

===>  EVAL Epoch 5


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 5 || Run Time:    4.1 | Load Time:    2.8 || F1:  94.35 | Prec:  96.69 | Rec:  92.12 || Ex/s: 359.33



94.34832756632065

### Amazon-Google

In [11]:
test_df = pd.read_csv('../Structured/Amazon-Google/merged_test.csv')
test_df.ltable_price = test_df.ltable_price.astype(str)
test_df.rtable_price = test_df.rtable_price.astype(str)
attackRandomAttribute(test_df,ignore_columns=['id','label'])
test_df.to_csv('../Structured/Amazon-Google/attacked_test_singleattr.csv',index=False)

In [14]:
train,validation,attacked_test = dm.data.process('../Structured/Amazon-Google',train='merged_train.csv',
                       validation='merged_valid.csv',test='attacked_test_singleattr.csv',left_prefix='ltable_',
                      right_prefix='rtable_',cache='attack_single.pth')


Reading and processing data from "../Structured/Amazon-Google/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Amazon-Google/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Amazon-Google/attacked_test_singleattr.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [15]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=10,best_save_path='../models/amazongoogle_hybrid_singleattack.pth',
              epochs=20)

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:24


Finished Epoch 1 || Run Time:   21.2 | Load Time:    3.5 || F1:  23.94 | Prec:  13.75 | Rec:  92.42 || Ex/s: 279.06

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    2.3 | Load Time:    1.1 || F1:  21.52 | Prec:  12.06 | Rec: 100.00 || Ex/s: 682.02

* Best F1: 21.51724137931034
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Finished Epoch 20 || Run Time:   19.8 | Load Time:    3.3 || F1:  96.28 | Prec:  92.83 | Rec: 100.00 || Ex/s: 297.59

===>  EVAL Epoch 20


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 20 || Run Time:    2.3 | Load Time:    1.0 || F1:  68.21 | Prec:  67.22 | Rec:  69.23 || Ex/s: 697.49

---------------------

Loading best model...
Training done.


70.45009784735811

In [16]:
model.run_eval(attacked_test)

===>  EVAL Epoch 11


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 11 || Run Time:    2.3 | Load Time:    1.2 || F1:  60.71 | Prec:  63.55 | Rec:  58.12 || Ex/s: 646.74



60.714285714285715

### Walmart-Amazon

In [17]:
test_df = pd.read_csv('../Structured/Walmart-Amazon/merged_test.csv')
test_df.ltable_price = test_df.ltable_price.astype(str)
test_df.rtable_price = test_df.rtable_price.astype(str)
attackRandomAttribute(test_df,ignore_columns=['id','label'])
test_df.to_csv('../Structured/Walmart-Amazon/attacked_test_singleattr.csv',index=False)

In [18]:
train,validation,attacked_test = dm.data.process('../Structured/Walmart-Amazon',train='merged_train.csv',
                       validation='merged_valid.csv',test='attacked_test_singleattr.csv',left_prefix='ltable_',
                      right_prefix='rtable_',cache='attack_single.pth')


Reading and processing data from "../Structured/Walmart-Amazon/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Walmart-Amazon/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../Structured/Walmart-Amazon/attacked_test_singleattr.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [19]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=10,best_save_path='../models/walmartamazon_hybrid_singleattack.pth',
              epochs=20)

* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 1 || Run Time:   28.7 | Load Time:    4.8 || F1:  22.18 | Prec:  12.60 | Rec:  92.19 || Ex/s: 183.66

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    3.4 | Load Time:    1.5 || F1:  28.18 | Prec:  16.65 | Rec:  91.71 || Ex/s: 412.70

* Best F1: 28.18471337579618
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 20 || Run Time:   28.7 | Load Time:    4.8 || F1:  99.14 | Prec:  98.29 | Rec: 100.00 || Ex/s: 183.52

===>  EVAL Epoch 20


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 20 || Run Time:    3.5 | Load Time:    1.5 || F1:  65.66 | Prec:  64.04 | Rec:  67.36 || Ex/s: 411.50

---------------------

Loading best model...
Training done.


65.65656565656566

In [20]:
model.run_eval(attacked_test)

===>  EVAL Epoch 19


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 19 || Run Time:    3.6 | Load Time:    1.7 || F1:  57.62 | Prec:  61.90 | Rec:  53.89 || Ex/s: 393.17



57.61772853185595

## Original results

In [4]:
itunes_amazon = dm.MatchingModel(attr_summarizer='hybrid')
itunes_amazon.load_state('../models/itunes_amazon_hybrid.pth')

In [8]:
train,test = dm.data.process('../Structured/itunes-amazon/',train='merged_train.csv',
                       test='merged_test.csv',left_prefix='ltable_',
                      right_prefix='rtable_')

In [9]:
itunes_amazon.run_eval(test)

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    0.7 | Load Time:    0.4 || F1:  81.48 | Prec:  81.48 | Rec:  81.48 || Ex/s: 108.32



81.48148148148148

In [4]:
beer = dm.MatchingModel(attr_summarizer='hybrid')
beer.load_state('../models/beer_hybrid.pth')

In [9]:
train,validation,test = dm.data.process('../Structured/Beer/',train='merged_train.csv',validation='merged_valid.csv',
                       test='merged_test.csv',left_prefix='ltable_',
                      right_prefix='rtable_')

In [10]:
beer.run_eval(test)

===>  EVAL Epoch 7
Finished Epoch 7 || Run Time:    0.2 | Load Time:    0.1 || F1:  75.00 | Prec:  66.67 | Rec:  85.71 || Ex/s: 263.50



75.0

In [11]:
dblpacm = dm.MatchingModel(attr_summarizer='hybrid')
dblpacm.load_state('../models/dblp_acm_hybrid.pth')

In [12]:
train,test = dm.data.process('../Structured/DBLP-ACM/',train='merged_train.csv',
                       test='merged_test.csv',left_prefix='ltable_',
                      right_prefix='rtable_')

In [13]:
dblpacm.run_eval(test)

===>  EVAL Epoch 7
Finished Epoch 7 || Run Time:    3.9 | Load Time:    3.9 || F1:  97.24 | Prec:  95.44 | Rec:  99.10 || Ex/s: 317.42



97.23756906077348

In [19]:
walmartamazon = dm.MatchingModel(attr_summarizer='hybrid')
walmartamazon.load_state('../models/walmartamazon_hybrid.pth')

In [15]:
train,test = dm.data.process('../Structured/Walmart-Amazon/',train='merged_train.csv',
                       test='merged_test.csv',left_prefix='ltable_',
                      right_prefix='rtable_')

In [20]:
walmartamazon.run_eval(test)

===>  EVAL Epoch 19
Finished Epoch 19 || Run Time:    4.5 | Load Time:    2.5 || F1:  60.25 | Prec:  58.91 | Rec:  61.66 || Ex/s: 293.85



60.25316455696202

In [21]:
amazongoogle = dm.MatchingModel(attr_summarizer='hybrid')
amazongoogle.load_state('../models/amazongoogle_hybrid.pth')

In [22]:
train,test = dm.data.process('../Structured/Amazon-Google/',train='merged_train.csv',
                       test='merged_test.csv',left_prefix='ltable_',
                      right_prefix='rtable_')

In [23]:
amazongoogle.run_eval(test)

===>  EVAL Epoch 9
Finished Epoch 9 || Run Time:    3.1 | Load Time:    1.8 || F1:  51.34 | Prec:  46.53 | Rec:  57.26 || Ex/s: 474.38



51.34099616858238

## Retrain models

### iTunes-Amazon

In [15]:
train_df = pd.read_csv('../Structured/itunes-amazon/merged_train.csv')
attackDataset(train_df,['id','label'])
train_df.to_csv('../Structured/itunes-amazon/attacked_train.csv',index=False)

In [21]:
valid_df = pd.read_csv('../Structured/itunes-amazon/merged_valid.csv')
attackDataset(valid_df,['id','label'])
valid_df.to_csv('../Structured/itunes-amazon/attacked_valid.csv',index=False)

In [22]:
train,validation,test = dm.data.process('../Structured/itunes-amazon/',train='attacked_train.csv',
                                       validation='attacked_valid.csv',test='attacked_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')

In [25]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=5,batch_size=16,epochs=20,
                best_save_path='../models/itunesamazon_hybrid_attacked.pth')

* Number of trainable parameters: 17757810
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:    7.5 | Load Time:    1.3 || F1:  42.34 | Prec:  27.05 | Rec:  97.44 || Ex/s:  36.41

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.9 | Load Time:    0.4 || F1:  60.76 | Prec:  46.15 | Rec:  88.89 || Ex/s:  87.08

* Best F1: 60.75949367088607
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:    7.8 | Load Time:    1.4 || F1:  56.57 | Prec:  41.04 | Rec:  91.03 || Ex/s:  34.88

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.8 | Load Time:    0.4 || F1:  65.38 | Prec:  68.00 | Rec:  62.96 || Ex/s:  86.43

* Best F1: 65.38461538461539
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:    8.0 | Load Time:    1.4 || F1:  60.00 | Prec:  43.60 | Rec:  96.15 || Ex/s:  34.47

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    0.8 | Load Time:    0.4 || F1:  65.62 | Prec:  56.76 | Rec:  77.

71.18644067796609

In [26]:
model.run_eval(test)

===>  EVAL Epoch 13
Finished Epoch 13 || Run Time:    0.6 | Load Time:    0.5 || F1:  69.70 | Prec:  58.97 | Rec:  85.19 || Ex/s:  97.59



69.6969696969697

### Beer

In [27]:
train_df = pd.read_csv('../Structured/Beer/merged_train.csv')
attackDataset(train_df,['id','label'])
train_df.to_csv('../Structured/Beer/attacked_train.csv',index=False)
valid_df = pd.read_csv('../Structured/Beer/merged_valid.csv')
attackDataset(valid_df,['id','label'])
valid_df.to_csv('../Structured/Beer/attacked_valid.csv',index=False)

In [28]:
train,validation,test = dm.data.process('../Structured/Beer/',train='attacked_train.csv',
                                       validation='attacked_valid.csv',test='attacked_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')

In [29]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=4,batch_size=16,epochs=20,
                best_save_path='../models/beer_hybrid_attacked.pth')

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:    2.1 | Load Time:    0.3 || F1:  34.62 | Prec:  28.12 | Rec:  45.00 || Ex/s: 108.28

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.3 | Load Time:    0.1 || F1:  75.00 | Prec:  90.00 | Rec:  64.29 || Ex/s: 215.80

* Best F1: 75.00000000000001
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:    2.2 | Load Time:    0.3 || F1:  67.29 | Prec:  53.73 | Rec:  90.00 || Ex/s: 104.93

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.3 | Load Time:    0.1 || F1:  74.07 | Prec:  76.92 | Rec:  71.43 || Ex/s: 211.91

---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:    2.2 | Load Time:    0.4 || F1:  80.00 | Prec:  69.09 | Rec:  95.00 || Ex/s: 103.07

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    0.3 | Load Time:    0.1 || F1:  75.86 | Prec:  73.33 | Rec:  78.57 || Ex/s: 206.41

* Best F1: 75.86206896551724
Saving b

78.57142857142857

In [30]:
model.run_eval(test)

===>  EVAL Epoch 9
Finished Epoch 9 || Run Time:    0.2 | Load Time:    0.1 || F1:  66.67 | Prec:  62.50 | Rec:  71.43 || Ex/s: 250.31



66.66666666666667

### Amazon-Google

In [31]:
train_df = pd.read_csv('../Structured/Amazon-Google/merged_train.csv')
attackDataset(train_df,['id','label'])
train_df.to_csv('../Structured/Amazon-Google/attacked_train.csv',index=False)
valid_df = pd.read_csv('../Structured/Amazon-Google/merged_valid.csv')
attackDataset(valid_df,['id','label'])
valid_df.to_csv('../Structured/Amazon-Google/attacked_valid.csv',index=False)

In [34]:
train,validation,test = dm.data.process('../Structured/Amazon-Google/',train='attacked_train.csv',
                                       validation='attacked_valid.csv',test='attacked_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')

In [35]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=10,epochs=20,
                best_save_path='../models/amazongoogle_hybrid_attacked.pth')

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:   34.6 | Load Time:    8.9 || F1:  21.23 | Prec:  11.97 | Rec:  93.71 || Ex/s: 158.00

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    3.1 | Load Time:    2.1 || F1:  22.98 | Prec:  13.10 | Rec:  93.59 || Ex/s: 446.84

* Best F1: 22.980062959076598
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:   35.6 | Load Time:    9.0 || F1:  29.34 | Prec:  17.46 | Rec:  91.85 || Ex/s: 154.17

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    3.1 | Load Time:    2.1 || F1:  25.24 | Prec:  14.56 | Rec:  94.87 || Ex/s: 439.01

* Best F1: 25.241614553723704
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:   35.3 | Load Time:    9.0 || F1:  39.62 | Prec:  25.01 | Rec:  95.28 || Ex/s: 155.25

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    3.1 | Load Time:    2.1 || F1:  34.76 | Prec:  22.08 | Rec:  81

43.30985915492957

In [36]:
model.run_eval(test)

===>  EVAL Epoch 8
Finished Epoch 8 || Run Time:    3.0 | Load Time:    2.1 || F1:  41.92 | Prec:  35.87 | Rec:  50.43 || Ex/s: 452.30



41.91829484902309

### DBLP-ACM

In [38]:
train_df = pd.read_csv('../Structured/DBLP-ACM/merged_train.csv')
attackDataset(train_df,['id','label'])
train_df.to_csv('../Structured/DBLP-ACM/attacked_train.csv',index=False)
valid_df = pd.read_csv('../Structured/DBLP-ACM/merged_validation.csv')
attackDataset(valid_df,['id','label'])
valid_df.to_csv('../Structured/DBLP-ACM/attacked_valid.csv',index=False)

In [7]:
train,validation,test = dm.data.process('../Structured/DBLP-ACM/',train='attacked_train.csv',
                                       validation='attacked_valid.csv',test='attacked_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')

In [6]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('../models/dblpacm_hybrid_attacked.pth')

In [43]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=10,epochs=15,
                best_save_path='../models/dblpacm_hybrid_attacked.pth')

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:   47.2 | Load Time:   19.3 || F1:  56.12 | Prec:  39.21 | Rec:  98.72 || Ex/s: 111.63

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    4.5 | Load Time:    4.7 || F1:  91.10 | Prec:  86.00 | Rec:  96.85 || Ex/s: 268.99

* Best F1: 91.10169491525424
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:   44.8 | Load Time:   18.9 || F1:  85.54 | Prec:  75.16 | Rec:  99.25 || Ex/s: 116.37

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    4.6 | Load Time:    4.7 || F1:  89.83 | Prec:  83.27 | Rec:  97.52 || Ex/s: 266.04

---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:   48.9 | Load Time:   18.7 || F1:  90.74 | Prec:  83.52 | Rec:  99.32 || Ex/s: 109.83

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    4.7 | Load Time:    4.8 || F1:  92.50 | Prec:  87.08 | Rec:  98.65 || Ex/s: 260.39

* Best F1: 92.50263991552269
Saving b

94.65648854961832

In [8]:
model.run_eval(test)

===>  EVAL Epoch 6
Finished Epoch 6 || Run Time:    4.4 | Load Time:    4.3 || F1:  94.78 | Prec:  91.60 | Rec:  98.20 || Ex/s: 284.32



94.78260869565216

### Walmart-Amazon

In [10]:
train_df = pd.read_csv('../Structured/Walmart-Amazon/merged_train.csv')
attackDataset(train_df,['id','label'])
train_df.to_csv('../Structured/Walmart-Amazon/attacked_train.csv',index=False)
valid_df = pd.read_csv('../Structured/Walmart-Amazon/merged_valid.csv')
attackDataset(valid_df,['id','label'])
valid_df.to_csv('../Structured/Walmart-Amazon/attacked_valid.csv',index=False)

In [14]:
train,validation,test = dm.data.process('../Structured/Walmart-Amazon/',train='attacked_train.csv',
                                       validation='attacked_valid.csv',test='attacked_test.csv',
                                       left_prefix='ltable_',right_prefix='rtable_')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.run_train(train,validation,pos_neg_ratio=10,epochs=20,
                best_save_path='../models/walmartamazon_hybrid_attacked.pth')

* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:   47.1 | Load Time:   11.8 || F1:  19.31 | Prec:  10.73 | Rec:  96.70 || Ex/s: 104.38

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    4.2 | Load Time:    2.9 || F1:  24.61 | Prec:  14.60 | Rec:  78.24 || Ex/s: 288.53

* Best F1: 24.612876935615326
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:   46.6 | Load Time:   11.5 || F1:  24.31 | Prec:  13.95 | Rec:  94.44 || Ex/s: 105.86

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    4.3 | Load Time:    2.8 || F1:  26.52 | Prec:  15.84 | Rec:  81.35 || Ex/s: 287.63

* Best F1: 26.52027027027027
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:   48.1 | Load Time:   11.5 || F1:  32.83 | Prec:  19.86 | Rec:  94.62 || Ex/s: 103.10

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    4.3 | Load Time:    2.8 || F1:  31.64 | Prec:  20.49 | Rec:  69

## Evaluated retrained on the old test set

In [15]:
from sklearn.metrics import f1_score,precision_score,recall_score

In [16]:
itunesamazon = dm.MatchingModel(attr_summarizer='hybrid')
itunesamazon.load_state('../models/itunes_amazon_forattack.pth')

In [20]:
def getPrecisionRecallF1(testpath,model):
    test = dm.data.process_unlabeled(testpath,model,
                                ignore_columns = ['id','label'])
    predictions = model.run_prediction(test,output_attributes =True)
    true_label = predictions.label
    predictions['predicted'] = predictions.match_score.apply(lambda m:round(m))
    y_pred = predictions.predicted
    return (f1_score(true_label,y_pred),precision_score(true_label,y_pred),
            recall_score(true_label,y_pred))

In [21]:
getPrecisionRecallF1('../Structured/itunes-amazon/merged_test.csv',
                    itunesamazon)

===>  PREDICT Epoch 7
Finished Epoch 7 || Run Time:    0.5 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(0.912280701754386, 0.8666666666666667, 0.9629629629629629)

In [22]:
beer = dm.MatchingModel(attr_summarizer='hybrid')
beer.load_state('../models/beer_hybrid_attacked.pth')

In [23]:
getPrecisionRecallF1('../Structured/Beer/merged_test.csv',beer)

===>  PREDICT Epoch 9
Finished Epoch 9 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(0.5714285714285714, 0.8571428571428571, 0.42857142857142855)

In [25]:
amazongoogle = dm.MatchingModel(attr_summarizer='hybrid')
amazongoogle.load_state('../models/amazongoogle_hybrid_attacked.pth')

In [26]:
getPrecisionRecallF1('../Structured/Amazon-Google/merged_test.csv',amazongoogle)

===>  PREDICT Epoch 8
Finished Epoch 8 || Run Time:    2.9 | Load Time:    1.6 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(0.5077720207253885, 0.4260869565217391, 0.6282051282051282)

In [28]:
walmartamazon = dm.MatchingModel(attr_summarizer='hybrid')
walmartamazon.load_state('../models/walmartamazon_hybrid_attacked.pth')

In [29]:
getPrecisionRecallF1('../Structured/Walmart-Amazon/merged_test.csv',walmartamazon)

===>  PREDICT Epoch 3
Finished Epoch 3 || Run Time:    4.3 | Load Time:    2.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(0.3409363745498199, 0.221875, 0.7357512953367875)

In [30]:
dblpacm = dm.MatchingModel(attr_summarizer='hybrid')
dblpacm.load_state('../models/dblpacm_hybrid_attacked.pth')

In [31]:
getPrecisionRecallF1('../Structured/DBLP-ACM/merged_test.csv',dblpacm)

===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    4.6 | Load Time:    4.2 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(0.9353128313891834, 0.8837675350701403, 0.9932432432432432)